In [89]:
import pandas as pd
import folium
import requests
from datetime import datetime
import ast
import io

In [90]:
#School Safety Zone
urlSchoolZone = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/642efeca-1258-4c23-8e86-d9affca26001/resource/b3268057-6b55-4e6e-b1ae-b752bce92a1c/download/Stationary%20Sign%20locations.geojson'
responseSchoolZone = requests.get(urlSchoolZone)
dataSchoolZone = responseSchoolZone.json()
dfSchoolZone = pd.json_normalize(dataSchoolZone, 'features')
dfSchoolZone = dfSchoolZone.drop('type', axis=1)


In [91]:
column_mapping = {}
for col in dfSchoolZone.columns:
    if col.startswith("properties."):
        new_col = col[len("properties."):]
        column_mapping[col] = new_col
dfSchoolZone.rename(columns=column_mapping, inplace=True)

In [92]:
#Traffic Collision
#There is Error decoding JSON data, so I use CSV file
urlTrafficCollision = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/ec53f7b2-769b-4914-91fe-a37ee27a90b3/resource/cb890861-ed20-4862-bb75-b1f9ec1e58dd/download/Traffic%20Collisions%20-%204326.csv'
responseTrafficCollision = requests.get(urlTrafficCollision)
csv_textTrafficCollision = responseTrafficCollision.text
dfTrafficCollision = pd.read_csv(io.StringIO(csv_textTrafficCollision))

In [93]:
#showing example of first 1000 data
dfTrafficCollision1000=dfTrafficCollision.iloc[0:1000]

In [94]:
coordinate=[]
for i in range(len(dfTrafficCollision1000)):
    coordinate.append(list(ast.literal_eval(dfTrafficCollision1000.iloc[i]["geometry"].replace("'", "\"")).values())[1])


In [95]:
dfTrafficCollision1000['coordinate'] = coordinate

/var/folders/fs/sg7k70td3m18zk_gxvll38nc0000gn/T/ipykernel_95189/1873428431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfTrafficCollision1000['coordinate'] = coordinate


In [96]:
#On-Street Permit Parking Area
urlOnStreetPark='https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/74382239-6e6b-4683-b2e7-b0ce86263b2b/resource/68109632-ad0f-4439-889e-941793c39606/download/On-Street%20Permit%20Parking%20Area%20Map%20Data%20-%204326.csv'
responseOnStreetPark= requests.get(urlOnStreetPark)
csv_textOnStreetPark = responseOnStreetPark.text
dfOnStreetPark = pd.read_csv(io.StringIO(csv_textOnStreetPark))

In [97]:
coordinate=[]
for i in range(len(dfOnStreetPark)):
    coordinate.append(list(ast.literal_eval(dfOnStreetPark.iloc[i]["geometry"].replace("'", "\"")).values())[1])
dfOnStreetPark['coordinate'] = coordinate

In [98]:
#On-Street Permit Parking Area
urlOnStreetPark='https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/74382239-6e6b-4683-b2e7-b0ce86263b2b/resource/68109632-ad0f-4439-889e-941793c39606/download/On-Street%20Permit%20Parking%20Area%20Map%20Data%20-%204326.csv'
responseOnStreetPark= requests.get(urlOnStreetPark)
csv_textOnStreetPark = responseOnStreetPark.text
dfOnStreetPark = pd.read_csv(io.StringIO(csv_textOnStreetPark))
coordinate=[]
for i in range(len(dfOnStreetPark)):
    coordinate.append(list(ast.literal_eval(dfOnStreetPark.iloc[i]["geometry"].replace("'", "\"")).values())[1])
dfOnStreetPark['coordinate'] = coordinate

In [99]:
coordinate=[]
for i in range(len(dfOnStreetPark)):
    coordinate.append(list(ast.literal_eval(dfOnStreetPark.iloc[i]["geometry"].replace("'", "\"")).values())[1])
dfOnStreetPark['coordinate'] = coordinate

In [100]:
#Automated Speed Enforcement Locations
urlSpeedEnforcement= 'https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/a154790c-4a8a-4d09-ab6b-535ddb646770/resource/c19bc58a-d034-4e04-bbc3-0ebdf73a5feb/download/Automated%20Speed%20Enforcement%20Locations%20-%204326.csv'
responseSpeedEnforcement = requests.get(urlSpeedEnforcement)
csv_textSpeedEnforcement = responseSpeedEnforcement.text
dfSpeedEnforcement = pd.read_csv(io.StringIO(csv_textSpeedEnforcement))

In [101]:
coordinate=[]
for i in range(len(dfSpeedEnforcement)):
    coordinate.append(list(ast.literal_eval(dfSpeedEnforcement.iloc[i]["geometry"].replace("'", "\"")).values())[1])
dfSpeedEnforcement['coordinate'] = coordinate


In [102]:
# Create a Folium map 
map = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

In [103]:
# Automated Speed Enforcement Locations plot
for index, row in dfSpeedEnforcement.iterrows():
    popup_text = "_id: {}<br>Location_Code	: {}<br>Ward: {}<br>Status: {}<br>location: {}<br>FID: {}<br>geometry: {}"\
    .format(row['_id'], row['Location_Code'], row['Ward'], row['Status'], row['location'], row['FID'], row['geometry'], 
    row['coordinate'][1], row['coordinate'][0])
    popup = folium.Popup(popup_text, max_width=300)
    icon_anchor= (0, 0)
    icon_url = 'https://www.toronto.ca/wp-content/uploads/2020/06/982d-TS_ASE_Sign-500x624.png'
    icon_size = (30, 30)  # Set the icon size in pixels
    icon = folium.features.CustomIcon(icon_url, icon_size, icon_anchor=icon_anchor)
    marker = folium.Marker(location=[row['coordinate'][1], row['coordinate'][0]], popup=popup_text, icon=icon)
    marker.add_to(map)
    # Create a new marker with the custom icon and popup text and add it to the map

In [104]:
# On Street Parking zone
for i in range(len(dfOnStreetPark)):
    dfOnStreetPark.at[i,'coordinate'] =[tuple(reversed(coord)) for coord in dfOnStreetPark.iloc[i, 20][0]]

for index, row in dfOnStreetPark.iterrows():
    popup_text = "_id: {}<br>AREA_ID: {}<br>DATE_EFFECTIVE: {}<br>DATE_EXPIRY: {}<br>AREA_ATTR_ID: {}<br>AREA_TYPE_ID: {}<br>PARENT_AREA_ID: {}<br>AREA_TYPE: {}<br>AREA_CLASS_ID: {}\
    <br>AREA_CLASS: {}<br>AREA_SHORT_CODE: {}<br>AREA_LONG_CODE: {}<br>AREA_NAME: {}<br>AREA_DESC: {}<br>FEATURE_CODE: {}<br>FEATURE_CODE_DESC: {}<br>TRANS_ID_CREATE: {}\
    <br>TRANS_ID_EXPIRE: {}<br>OBJECTID: {}<br>geometry: {}"\
    .format(row['_id'], row['AREA_ID'], row['DATE_EFFECTIVE'], row['DATE_EXPIRY'], row['AREA_ATTR_ID'], row['AREA_TYPE_ID'], row['PARENT_AREA_ID'], row['AREA_TYPE'], \
    row['AREA_CLASS_ID'], row['AREA_CLASS'], row['AREA_SHORT_CODE'], row['AREA_LONG_CODE'], row['AREA_NAME'], row['AREA_DESC'], row['FEATURE_CODE'],row['AREA_SHORT_CODE'], row['TRANS_ID_CREATE'], \
    row['TRANS_ID_CREATE'], row['OBJECTID'], row['geometry'])
    
    popup = folium.Popup(popup_text, max_width=300)
    polygon = folium.Polygon(locations=dfOnStreetPark.iloc[index,20], color='blue', fill_color='green', fill_opacity=0.4)
    polygon.add_to(map)
    polygon.add_child(popup)
    ''''
    icon_anchor= (0, 0)
    icon_url = 'https://cdn.pixabay.com/photo/2017/10/18/17/37/sign-2864889_1280.png'
    icon_size = (30, 30)  # Set the icon size in pixels
    icon = folium.features.CustomIcon(icon_url, icon_size, icon_anchor=icon_anchor)
    marker = folium.Marker(location=[row['coordinate'][1], row['coordinate'][0]], popup=popup_text, icon=icon)
    
    
    geojson = folium.GeoJson(data=dfOnStreetPark.iloc[index, 20], style_function=lambda x: {'fillColor': 'yellow', 'color': 'black', 'weight': 1},\
    highlight_function=lambda x: {'fillColor': 'green', 'color': 'blue', 'weight': 3},\
    tooltip=folium.GeoJsonTooltip(fields=['popup_text'], aliases=['Name:']))
    geojson.add_to(map)
    '''
    # Create a new marker with the custom icon and popup text and add it to the map

In [105]:
# Traffic Collisions plot
for index, row in dfTrafficCollision1000.iterrows():
    popup_text = "_id: {}<br>EventUniqueId: {}<br>OccurrenceDate: {}<br>Month: {}<br>Day_of_Week: {}<br>Year: {}<br>Hour: {}<br>Division: {}<br>Atom: {}\
    <br>Neighbourhood: {}<br>Fatalities: {}<br>Injury_Collisions: {}<br>FTR_Collisions: {}<br>PD_Collisions: {}<br>Latitude: {}<br>Longitude: {}"\
    .format(row['_id'], row['EventUniqueId'], row['OccurrenceDate'], row['Month'], row['Day_of_Week'], row['Year'], row['Hour'], row['Division'], \
    row['Atom'], row['Neighbourhood'], row['Fatalities'], row['Injury_Collisions'], row['FTR_Collisions'], row['PD_Collisions'],\
    row['coordinate'][1], row['coordinate'][0])
    popup = folium.Popup(popup_text, max_width=300)
    icon_anchor= (0, 0)
    icon_url = 'https://cdn.pixabay.com/photo/2017/10/18/17/37/sign-2864889_1280.png'
    icon_size = (30, 30)  # Set the icon size in pixels
    icon = folium.features.CustomIcon(icon_url, icon_size, icon_anchor=icon_anchor)
    marker = folium.Marker(location=[row['coordinate'][1], row['coordinate'][0]], popup=popup_text, icon=icon)
    marker.add_to(map)
    # Create a new marker with the custom icon and popup text and add it to the map

In [106]:
# School safety zone plot
# Add markers for each address to the map    popup_text = "Address: {}<br>Latitude: {}<br>Longitude: {}".format(row['Address'], row['Latitude'], row['Longitude'])
for index, row in dfSchoolZone.iterrows():
    popup_text = "_id: {}<br>address: {}<br>dir: {}<br>end_date: {}<br>flash_speed: {}<br>min_speed: {}<br>schedule: {}<br>sign_id: {}<br>sign_name: {}\
    <br>speed_limit: {}<br>start_date: {}<br>strobe_speed: {}<br>ward_no: {}<br>geometry.type: {}<br>Latitude: {}<br>Longitude: {}"\
    .format(row['_id'], row['address'], row['dir'], row['end_date'], row['flash_speed'], row['min_speed'], row['schedule'], row['sign_id'], 
    row['sign_name'], row['speed_limit'], row['start_date'], row['strobe_speed'], row['ward_no'], row['geometry.type'],
    row['geometry.coordinates'][1], row['geometry.coordinates'][0])
    popup = folium.Popup(popup_text, max_width=300)
    icon_anchor= (0, 0)
    icon_url = 'https://stinson.ca/media/catalog/product/cache/4425efb2f927b0d14b906c7707771a80/w/c/wc-1_school_area.png' 
    icon_size = (30, 30)  # Set the icon size in pixels
    icon = folium.features.CustomIcon(icon_url, icon_size, icon_anchor=icon_anchor)
    # Create a new marker with the custom icon and popup text and add it to the map
    marker = folium.Marker(location=[row['geometry.coordinates'][1], row['geometry.coordinates'][0]], popup=popup_text, icon=icon)
    marker.add_to(map)


In [107]:
map